# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos: sistema LLM + RAG**

#### **Nombres y matrículas de los integrantes del equipo:**

**Equipo 42**

|  NOMBRE COMPLETO             |     MATRÍCULA     | 
| :--------------------------: |:-----------------:|
| Alejandro Díaz Villagómez    |  A01276769        | 
| Emiliano Saucedo Arriola     |  A01659258        | 
| Javier Adrian Villa León     |  A01242469        | 
| Karina Zecua Cruz            |  A01795651        | 

* ##### **El formato de este cuaderno de Jupyter es libre, pero debe incuir al menos las siguientes secciones:**

  * ##### **Introducción de la problemática a resolver.**
  * ##### **Sistema RAG + LLM**
  * ##### **El chatbot, incluyendo ejemplos de prueba.**
  * ##### **Conclusiones**

* ##### **Pueden importar los paquetes o librerías que requieran.**

* ##### **Pueden incluir las celdas y líneas de código que deseen.**

# **Problemática a resolver**

Introducción y justificación donde se describa la temática elegida y la
problemática que el chatbot busca resolver o apoyar

# **Sistema RAG + LLM**

Informe documentando de la manera en que funciona el chatbot con un sistema RAG + LLM

# **Chatbot (con ejemplos)**

```
ollama run gemma3:1b
```

```
ollama run gemma2:2b
```

```
/bye
```

In [1]:
# Instalaciones necesarias
# !pip install langchain langchain-community langchain-ollama chromadb sentence-transformers faiss-cpu pypdf python-docx gradio
# !pip freeze > requirements.txt

In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
import json
from pathlib import Path
from typing import List, Dict, Any, Optional

import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    DirectoryLoader,
    Docx2txtLoader,
)
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS, Chroma
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.schema import Document

In [3]:
class DocumentLoader:
    def __init__(self):
        pass

    def load(self, paths: List[str]) -> List[Document]:
        all_docs = []
        for path in paths:
            path_obj = Path(path)
            if path_obj.is_file():
                all_docs.extend(self._load_file(str(path_obj)))
            elif path_obj.is_dir():
                all_docs.extend(self._load_dir(str(path_obj)))
        return all_docs

    def _load_file(self, file_path: str) -> List[Document]:
        ext = Path(file_path).suffix.lower()
        loader = {
            ".pdf": PyPDFLoader,
            ".txt": lambda p: TextLoader(p, encoding="utf-8"),
            ".docx": Docx2txtLoader,
            ".doc": Docx2txtLoader,
        }.get(ext, lambda p: TextLoader(p, encoding="utf-8"))
        return loader(file_path).load()

    def _load_dir(self, dir_path: str) -> List[Document]:
        docs = []
        for ext, loader_cls in {
            "**/*.pdf": PyPDFLoader,
            "**/*.txt": TextLoader,
            "**/*.docx": Docx2txtLoader,
        }.items():
            try:
                loader = DirectoryLoader(dir_path, glob=ext, loader_cls=loader_cls)
                docs.extend(loader.load())
            except:
                pass
        return docs

In [4]:
class VectorStoreManager:
    def __init__(
        self, embedding_model, db_type="faiss", chunk_size=1000, chunk_overlap=200
    ):
        self.embedding_model = embedding_model
        self.db_type = db_type
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.vector_store = None
        self.retriever = None

    def process_and_store(
        self, documents: List[Document], persist_path: Optional[str] = None
    ):
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            length_function=len,
        )
        chunks = splitter.split_documents(documents)
        if self.db_type == "faiss":
            self.vector_store = FAISS.from_documents(chunks, self.embedding_model)
        elif self.db_type == "chroma":
            self.vector_store = Chroma.from_documents(
                chunks, self.embedding_model, persist_directory=persist_path
            )
        else:
            raise ValueError("Tipo de vector store no soportado")
        self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})

In [5]:
class RAGChatbot:
    def __init__(
        self,
        model_name="gemma3:1b",
        embedding_model="gemma2:2b",
        db_type="faiss",
        **kwargs,
    ):
        self.llm = OllamaLLM(
            model=model_name, 
            temperature=kwargs.get("temperature", 0.3)
        )
        self.embeddings = OllamaEmbeddings(model=embedding_model)
        self.memory = ConversationBufferWindowMemory(
            k=5, 
            return_messages=True, 
            memory_key="chat_history", 
            output_key="answer"
        )
        self.loader = DocumentLoader()
        self.vector_mgr = VectorStoreManager(
            self.embeddings, 
            db_type=db_type, 
            **kwargs
        )
        self.qa_chain = None

        self.prompt = PromptTemplate(
            template=self._default_prompt(),
            input_variables=["context", "question", "chat_history"],
        )

    def setup(self, document_paths: List[str], persist_dir: Optional[str] = None):
        documents = self.loader.load(document_paths)
        self.vector_mgr.process_and_store(documents, persist_dir)
        self.setup_qa_chain()

    def setup_qa_chain(self):
        if not self.vector_mgr.retriever:
            raise ValueError("El retriever no está configurado")
        
        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.vector_mgr.retriever,
            memory=self.memory,
            return_source_documents=True,
            combine_docs_chain_kwargs={"prompt": self.prompt},
        )

    def ask(self, question: str) -> Dict[str, Any]:
        if not self.qa_chain:
            raise ValueError("La cadena QA no está configurada")
        try:
            result = self.qa_chain.invoke({"question": question})
            return {
                "question": question,
                "answer": result["answer"],
                "sources": [
                    doc.metadata.get("source", "Unknown")
                    for doc in result.get("source_documents", [])
                ],
                "source_documents": result.get("source_documents", []),
            }
        except Exception as e:
            import traceback

            traceback.print_exc()
            return {
                "question": question,
                "answer": f"Error al procesar la pregunta: {str(e)}",
                "sources": [],
                "source_documents": [],
            }

    def export_chat_history(self, json_path="chat_history/json_format.json"):
        os.makedirs(Path(json_path).parent, exist_ok=True)  # Crear carpeta si no existe
        chat = self.memory.chat_memory.messages
        json_data = [{"role": msg.type, "content": msg.content} for msg in chat]

        with open(json_path, "w", encoding="utf-8") as jf:
            json.dump(json_data, jf, ensure_ascii=False, indent=2)

        return f"📝 Historial exportado a: {Path(json_path).resolve()}"

    def _default_prompt(self):
        return """
            Eres un asistente útil que responde preguntas basadas en el contexto proporcionado y el historial de la conversación.
            Responde siempre en español.

            Historial de Conversación:
            {chat_history}

            Contexto del Documento:
            {context}

            Pregunta:
            {question}

            Respuesta útil en español:
        """

In [6]:
# Ruta a los documentos
document_paths = ["./documents"]

chatbot = RAGChatbot(
    model_name="gemma3:1b",
    embedding_model="gemma2:2b",
    db_type="faiss",
    chunk_size=1000,
    chunk_overlap=200,
)

# Configurar el sistema completo
chatbot.setup(document_paths=document_paths)

/var/folders/zb/tl73nj3d2933b0zx9fk5tfbm0000gn/T/ipykernel_17207/896455901.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferWindowMemory(


In [7]:
respuesta = chatbot.ask("¿Qué información contiene el documento?")
print("🧠 Respuesta:", respuesta["answer"])
print("📚 Fuentes:", respuesta["sources"])

🧠 Respuesta: El documento trata sobre el Proyecto 'Titán', que es una iniciativa de desarrollo de software para la gestión de clientes (CRM). Fue aprobado en la reunión del Q3 de 2024 y sus características principales son: Módulo de Autenticación (con OAuth 2.0 y 2FA), Módulo de Clientes (con historial de interacciones), y Módulo de Reportes (generación de informes de ventas mensuales en PDF y CSV). El líder técnico es Ana García y el equipo de desarrollo incluye Juan Pérez (Backend) y María López (Frontend). El presupuesto asignado para el MVP es de $50,000 USD.

📚 Fuentes: ['documents/sample.txt']


In [8]:
respuesta = chatbot.ask("Te acuerdas cuál fue mi pregunta anterior?")
print("🧠 Respuesta:", respuesta["answer"])
print("📚 Fuentes:", respuesta["sources"])

🧠 Respuesta: Mi pregunta anterior era: ¿Qué información contiene el documento?

📚 Fuentes: ['documents/sample.txt']


In [9]:
respuesta = chatbot.ask("Y quién era el líder del proyecto?")
print("🧠 Respuesta:", respuesta["answer"])
print("📚 Fuentes:", respuesta["sources"])

🧠 Respuesta: El líder del proyecto era Ana García.
📚 Fuentes: ['documents/sample.txt']


In [10]:
respuesta = chatbot.ask("Te acuerdas cuál fue tu última respuesta?")
print("🧠 Respuesta:", respuesta["answer"])
print("📚 Fuentes:", respuesta["sources"])

🧠 Respuesta: El líder del proyecto era Ana García.
📚 Fuentes: ['documents/sample.txt']


In [11]:
respuesta = chatbot.ask("Qué es Google?")
print("🧠 Respuesta:", respuesta["answer"])
print("📚 Fuentes:", respuesta["sources"])

🧠 Respuesta: Google es una empresa tecnológica multinacional con sede en Silicon Valley, que se especializa en búsqueda en línea, sistemas operativos, herramientas de productividad y computación en la nube. Fundada en 1996, Google es una de las empresas más valiosas del mundo.

📚 Fuentes: ['documents/sample.txt']


# **Gradio**

In [12]:
def gradio_interface(chatbot: RAGChatbot):
    def respond(message, history):
        response = chatbot.ask(message)
        return response["answer"]

    # Esta función ahora solo llama a export_chat_history y el gr.Info() se encargará del toast
    def export_chat_action():
        try:
            message = chatbot.export_chat_history()
            gr.Info(message)
        except Exception as e:
            error_message = f"Error al exportar historial: {str(e)}"
            gr.Warning(error_message)
        return ""

    # Definimos un tema personalizado para Gradio
    custom_theme = gr.themes.Soft(
        primary_hue="purple",
        secondary_hue="cyan",
        neutral_hue="gray",
    )

    with gr.Blocks(theme=custom_theme) as demo:
        # Inyectamos CSS para asegurar que el texto sea legible
        gr.HTML(
            """
            <style>
                body { background-color: var(--background-fill-primary); }
                .gradio-container { background-color: var(--background-fill-primary); }
                .message.user .wrap {
                    background-color: var(--chatbot-bubble-user-background-color) !important;
                    color: black !important;
                }
                .message.bot .wrap {
                    background-color: var(--chatbot-bubble-bot-background-color) !important;
                    color: black !important;
                }

                .gr-form input[type="text"], .gr-form textarea {
                    color: black !important;
                }
            </style>
            """
        )

        gr.Markdown(
            """
            # 📚 Chatbot RAG con Memoria
            ¡Hola! Soy tu asistente basado en documentos. Hazme preguntas sobre el Proyecto 'Titán'.
            """
        )

        # El ChatInterface ya incluye su propia caja de texto para entrada
        chatbot_ui = gr.ChatInterface(
            fn=respond,
            title=" ",  # Título ya lo pusimos en Markdown, o puedes ponerlo aquí
            description=" ",  # Descripción ya la pusimos en Markdown, o puedes ponerla aquí
        )

        with gr.Row():
            exportar_btn = gr.Button("📁 Exportar Historial de Chat")
            # Este Textbox ya no necesita ser actualizado directamente por el botón,
            # lo mantenemos por si acaso o para otros usos, pero no lo actualizaremos con el toast
            # gr.Info se encarga de eso.
            export_result_display = gr.Textbox(
                label="Estado de Exportación", interactive=False, visible=False
            )

        # El botón de exportar llama a la función que dispara el toast
        exportar_btn.click(fn=export_chat_action, outputs=None)

    return demo

In [13]:
# Ruta a los documentos
document_paths = ["./documents"]

new_chatbot = RAGChatbot(
    model_name="gemma3:1b",
    embedding_model="gemma2:2b",
    db_type="faiss",
    chunk_size=1000,
    chunk_overlap=200,
)

# Configurar el sistema completo
new_chatbot.setup(document_paths=document_paths)

demo = gradio_interface(new_chatbot)
demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


# **Conclusiones:**

* #### **Incluyan sus conclusiones de la actividad chatbot LLM + RAG:**



None

# **Fin de la actividad chatbot: LLM + RAG**